<a href="https://colab.research.google.com/github/annabelcoates/redev_machine_learning_samples/blob/master/FootballPredicator/fp_fetch_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install packages

In [0]:
!pip install PyGithub
!pip install import-ipynb
!pip install -U -q PyDrive


# Import Libraries

In [0]:
import sys
import os;
import datetime;
import pandas as pd
import numpy as np
import urllib.request
from github import Github
import import_ipynb

# Define `username` and `password` Variables

In [0]:
username='annabelcoates'
password='tammy 4 PRES'

# Define Fetch Functions

In [0]:
dateparselambda  = lambda x : (read_csv_date(x))

def read_csv_date( x ):
    try:
        if len(x) > 8:
            return pd.datetime.strptime(x, "%d/%m/%Y")
        else:
            return pd.datetime.strptime(x, "%d/%m/%y")
    except:
        return datetime.datetime(1970,1,1,0,0)

def get_raw_dataframe():
  url_base='https://raw.githubusercontent.com/RedevLtd/ml-colab-test-1/master/football_data/' # Read from github repository 
  start_year=0
  end_year=19
  leagues = ["E0", "E1", "E2", "E3"]
  #name_list=['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','Attendance',
  #            'Referee','HS','AS','HST','AST','HHW','AHW','HC','AC','HF','AF','HO','AO','HY','AY','HR',
  #            'AR','HBP','ABP','GBH','GBD','GBA','IWH','IWD','IWA','LBH','LBD','LBA','SBH','SBD','SBA',
 #             'SYH',	'SYD',	'SYA','WHH','WHD','WHA']
  fields_attendance=["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR","Attendance","WHH","WHD","WHA"]
  fields=["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR","WHH","WHD","WHA"]
  df=pd.DataFrame()

  for year in range(start_year,end_year):
    season= format( year, '02d') + format( year+1, '02d')
    for league in leagues:
      url=url_base+ league +'_'+str(season)+'.csv'
      try:
        temp_df= pd.read_csv( url, error_bad_lines = True, keep_default_na = True,
                      na_filter = True, skip_blank_lines = False,
                      parse_dates = ["Date"], date_parser=dateparselambda,
                      encoding = "ISO-8859-1", usecols=fields_attendance)
      except: # If it can't find the attendance column
        temp_df=pd.read_csv( url, error_bad_lines = True, keep_default_na = True,
                      na_filter = True, skip_blank_lines = False,
                      parse_dates = ["Date"], date_parser=dateparselambda,
                      encoding = "ISO-8859-1", usecols=fields)
      df=pd.concat([df,temp_df],axis=0,sort=False)

  df.reset_index(inplace=True)
  df=df.drop(['index'],axis=1)
  return df


# Define preprocessing functions

In [0]:
def season_year( row ) : 
    year = row.MatchDate.year
    month = row.MatchDate.month
    if month < 8 :
        return year - 1;
    else:
        return year;

In [0]:
def correct_football_dataframe( df ): 
    "corrects the data in the dataframe"
    # Full column names
    df.rename(columns={
                       "Date": "MatchDate",
                       "FTHG": "L_FTHomeGoals",
                       "FTAG": "L_FTAwayGoals",
                       "FTR":  "L_FTResult",
                       "HTHG": "L_HTHomeGoals",
                       "HTAG": "L_HTAwayGoals",
                       "HTR":  "L_HTResult",
                       "WHH":  "OddsHomeWin",    # Taking odds from William Hill as consistant in all years dataset
                       "WHD": "OddsDraw",
                       "WHA":  "OddsAwayWin",
                       }, inplace=True)
    
    
    # Add extra label data (breaking down some complex types to simple labels)
    df["SeasonYear"] = df.apply( lambda row: season_year(row), axis=1)
    df["SeasonMatchNo"] = range( 1, len(df) + 1 )
    df["OddsWinDiff"] = df.apply( lambda row: abs(row.OddsHomeWin-row.OddsAwayWin), axis=1)
    df["MatchNo"] = df.apply( lambda row: row.SeasonYear*10000+row.SeasonMatchNo, axis=1)
    df["FDIndex"] = df.apply( lambda row: str(row.Div)+str(row.MatchNo), axis=1)
    df.set_index( "FDIndex", inplace=True)
    
    # Correct some misspelt teams
    df["HomeTeam"] = df["HomeTeam"].apply( lambda team: "Middlesbrough" if team == "Middlesboro" else team )
    df["AwayTeam"] = df["AwayTeam"].apply( lambda team: "Middlesbrough" if team == "Middlesboro" else team )
  
    # Add labels
    df["L_FTHomeWin"] = df.apply(lambda row: 1 if row.L_FTResult == "H" else 0, axis=1)
    df["L_FTDraw"]    = df.apply(lambda row: 1 if row.L_FTResult == "D" else 0, axis=1)
    df["L_FTAwayWin"] = df.apply(lambda row: 1 if row.L_FTResult == "A" else 0, axis=1)
    df["L_HTHomeWin"] = df.apply(lambda row: 1 if row.L_HTResult == "H" else 0, axis=1)
    df["L_HTDraw"]    = df.apply(lambda row: 1 if row.L_HTResult == "D" else 0, axis=1)
    df["L_HTAwayWin"] = df.apply(lambda row: 1 if row.L_HTResult == "A" else 0, axis=1)
    df["Attendance"]=df["Attendance"]
    
    #df = add_football_features( df )
  
    return df  



In [0]:
def remove_null_lines(df):
  "Removes null lines of data which are created between the raw data files"
  df=df[df.MatchDate!=datetime.datetime(1970,1,1)]#"1970-01-01"
  return df

In [0]:
def preprocess () :
    'processes the specified raw directory path and puts the results in the results file'
    df = get_raw_dataframe() # This function comes from the fetcher code that is executed at the beginning
    df.fillna(0)
    df= correct_football_dataframe(df)
    df=remove_null_lines(df)
    #df.sort_values( by = ["MatchDate", "Div"], ascending = [1,1], inplace = True)
    df = df[[  "Div", "MatchNo", "SeasonYear", "SeasonMatchNo", "MatchDate","HomeTeam","AwayTeam","OddsHomeWin","OddsDraw","OddsAwayWin",
             "OddsWinDiff","Attendance","L_FTHomeGoals","L_FTAwayGoals","L_FTResult","L_FTHomeWin","L_FTDraw", "L_FTAwayWin",
             "L_HTHomeGoals","L_HTAwayGoals","L_HTResult", "L_HTHomeWin","L_HTDraw", "L_HTAwayWin"]]

    return df
  

In [0]:
def write_preprocessed_dataframe(df, username=username, password=password):
  'writes the dataframe to a csv file in the redev_machine_learning_samples repository, cannot overwrite existing files'
  # First write to a local csv temporarily
  csv_text=df.to_csv(sep=',',na_rep=0,index=False)
  # Log onto github and upload a new csv file
  path='FootballPredicator/preprocessed_data/preprocessed_data.csv'

  g=Github(username,password)
  user=g.get_user()
  redev=g.get_organization('RedevLtd')
  repo=redev.get_repo("redev_machine_learning_samples") # hardcoded for now
  try:
    repo.create_file(path=path,message='Created preprocessed_data.csv',content=csv_text)
    print('Uploaded to github')
  except:
    print('File already exists')
  return 

Create a dataframe with processed data

In [0]:
df=preprocess()


Display the data in a table

In [0]:
df.head(3)

,Div,MatchNo,SeasonYear,SeasonMatchNo,MatchDate,HomeTeam,AwayTeam,OddsHomeWin,OddsDraw,OddsAwayWin,OddsWinDiff,Attendance,L_FTHomeGoals,L_FTAwayGoals,L_FTResult,L_FTHomeWin,L_FTDraw,L_FTAwayWin,L_HTHomeGoals,L_HTAwayGoals,L_HTResult,L_HTHomeWin,L_HTDraw,L_HTAwayWin
FDIndex,,,,,,,,,,,,,,,,,,,,,,,,
E020000001,E0,20000001,2000,1,2000-08-19,Charlton,Man City,2.10,3.2,3.10,1.00,20043.0,4.0,0.0,H,1,0,0,2.0,0.0,H,1,0,0
E020000002,E0,20000002,2000,2,2000-08-19,Chelsea,West Ham,1.44,3.6,6.50,5.06,34914.0,4.0,2.0,H,1,0,0,1.0,0.0,H,1,0,0
E020000003,E0,20000003,2000,3,2000-08-19,Coventry,Middlesbrough,2.30,3.2,2.62,0.32,20624.0,1.0,3.0,A,0,0,1,1.0,1.0,D,0,1,0


Write the dataframe to a csv file on github

In [0]:
write_preprocessed_dataframe(df)
# Cannot overwrite a csv file with the same name

Uploaded to github
